# Google Colab Rust Setup

The following cell is used to set up and spin up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 

In [ ]:
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Network Programming with Rust

### Building Networked Applications with Rust

In [10]:
use std::process::{Command, Output, Stdio};

// A helper function to execute a shell command from a Rust script
fn execute_command(command: &str) -> Result<(), std::io::Error> {
    let status = Command::new("bash")
        .arg("-c")
        .arg(command)
        .stderr(Stdio::inherit())
        .status()?;

    if status.success() {
        Ok(())
    } else {
        Err(std::io::Error::from_raw_os_error(status.code().unwrap_or(1)))
    }
}

In [2]:
use std::net::{TcpListener, TcpStream};
use std::io::{Read, Write};

fn handle_client(mut stream: TcpStream) {
    println!("Handling client connection...");

    let mut buffer = [0; 512];
    stream.read(&mut buffer).unwrap();

    let request = String::from_utf8_lossy(&buffer[..]);
    println!("Received request:\n{}", request);

    let response = b"HTTP/1.1 200 OK\r\n\r\nHello, Rust!";
    stream.write(response).unwrap();

    println!("Response sent.");
}

let listener = TcpListener::bind("127.0.0.1:8080").unwrap();
println!("Server listening on 127.0.0.1:8080...");

for stream in listener.incoming() {
    match stream {
        Ok(stream) => {
            println!("Accepted a new connection");
            std::thread::spawn(|| handle_client(stream));
        }
        Err(e) => {
            eprintln!("Error: {}", e);
        }
    }
}

Server listening on 127.0.0.1:8080...
Accepted a new connection
Handling client connection...
Received request:
GET / HTTP/1.1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
Response sent.


Error: Subprocess terminated with status: signal: 9 (SIGKILL)

In [5]:
:dep tokio = {version = "1.35.0", features=["full"]}

In [ ]:
let command = "cd 1-building-networked-applications && cargo run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/building-networked-applications`


Server listening on 127.0.0.1:8080...
Accepted a new connection
Received request:
GET / HTTP/1.1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
Response sent.


### Transmission Control Protocol (TCP)

In [4]:
use std::net::{TcpListener, TcpStream};
use std::io::{Read, Write};

fn handle_tcp_client(mut stream: TcpStream) {
    let mut buffer = [0; 512];
    stream.read(&mut buffer).unwrap();

    let request = String::from_utf8_lossy(&buffer);
    println!("Received TCP request: {}", request);

    let response = b"HTTP/1.1 200 OK\r\n\r\nHello, TCP!";
    stream.write(response).unwrap();
}

let listener = TcpListener::bind("127.0.0.1:8080").unwrap();
for stream in listener.incoming() {
    match stream {
        Ok(stream) => {
            std::thread::spawn(|| handle_tcp_client(stream));
        }
        Err(e) => { 
            eprintln!("Error: {}", e);
        }
    }
}

Received TCP request: GET / HTTP/1.1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 


Error: Subprocess terminated with status: signal: 9 (SIGKILL)

### User Datagram Protocol (UDP)

In [ ]:
use std::net::UdpSocket;

fn handle_udp_client(socket: UdpSocket) {
    let mut buffer = [0; 512];

    let (_, client_address) = socket.recv_from(&mut buffer).unwrap();

    let request = String::from_utf8_lossy(&buffer);
    println!("Received UDP request from {}: {}", client_address, request);

    let response = b"Hello, UDP!";
    socket.send_to(response, client_address).unwrap();
}

let socket = UdpSocket::bind("127.0.0.1:8080").unwrap();

loop {
    handle_udp_client(socket.try_clone().unwrap());
}

Received UDP request from 127.0.0.1:44881: Test message                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    


### Asynchronous Network Operations

#### Asynchronous Programming in Rust

In [4]:
:dep reqwest = { version = "0.11.22" }

In [5]:
use reqwest::Client;

async fn fetch_data() -> Result<String, reqwest::Error> {
    let url = "https://jsonplaceholder.typicode.com/posts";

    let client = Client::new();
    let response = client.get(url).send().await?;

    let body = response.text().await?;

    Ok(body)
}

#[tokio::main]
async fn main() {
    match fetch_data().await {
        Ok(data) => println!("Fetched data: {}", data),
        Err(e) => eprintln!("Error fetching data: {}", e),
    }
}

main()

Fetched data: [
  {
    "userId": 1,
    "id": 1,
    "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
    "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
  },
  {
    "userId": 1,
    "id": 2,
    "title": "qui est esse",
    "body": "est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla"
  },
  {
    "userId": 1,
    "id": 3,
    "title": "ea molestias quasi exercitationem repellat qui ipsa sit aut",
    "body": "et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut"
  },
  {
    "userId": 1,
    "id": 4,
    "title": "eum et est occaecati",
    "body": "ullam

()

    "body": "reprehenderit quos placeat\nvelit minima officia dolores impedit repudiandae molestiae nam\nvoluptas recusandae quis delectus\nofficiis harum fugiat vitae"
  },
  {
    "userId": 2,
    "id": 16,
    "title": "sint suscipit perspiciatis velit dolorum rerum ipsa laboriosam odio",
    "body": "suscipit nam nisi quo aperiam aut\nasperiores eos fugit maiores voluptatibus quia\nvoluptatem quis ullam qui in alias quia est\nconsequatur magni mollitia accusamus ea nisi voluptate dicta"
  },
  {
    "userId": 2,
    "id": 17,
    "title": "fugit voluptas sed molestias voluptatem provident",
    "body": "eos voluptas et aut odit natus earum\naspernatur fuga molestiae ullam\ndeserunt ratione qui eos\nqui nihil ratione nemo velit ut aut id quo"
  },
  {
    "userId": 2,
    "id": 18,
    "title": "voluptate et itaque vero tempora molestiae",
    "body": "eveniet quo quis\nlaborum totam consequatur non dolor\nut et est repudiandae\nest voluptatem vel debitis et magnam"
  },
  {
    "us

#### Asynchronous TCP Server

In [ ]:
use tokio::net::TcpListener;
use tokio::io::{AsyncReadExt, AsyncWriteExt};

async fn handle_async_tcp_client(mut socket: tokio::net::TcpStream) {
    let mut buffer = [0; 512];
    let bytes_read = socket.read(&mut buffer).await.unwrap();

    let request = String::from_utf8_lossy(&buffer[..bytes_read]);
    println!("Received asynchronous TCP request: {}", request);

    let response = b"HTTP/1.1 200 OK\r\n\r\nHello, Asynchronous TCP!";
    socket.write_all(response).await.unwrap();
}

#[tokio::main]
async fn main() {
    let listener = TcpListener::bind("127.0.0.1:8080").await.unwrap();

    while let Ok((stream, _)) = listener.accept().await {
        tokio::spawn(handle_async_tcp_client(stream));
    }
}

main()

Received asynchronous TCP request: GET / HTTP/1.1



In [ ]:
let command = "cd 3-asynchronous-tcp-server && cargo run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/asynchronous-tcp-server`


Received asynchronous TCP request: GET / HTTP/1.1



### Real-World Use Cases in Network Programming

#### Handle HTTP requests with Hyper

In [4]:
:dep hyper = { version = "0.14.27", features = ["http1", "server", "tcp"] }

In [ ]:
use hyper::service::{make_service_fn, service_fn};
use hyper::{Body, Request, Response, Server};

async fn handle_request(_: Request<Body>) -> Result<Response<Body>, hyper::Error> {
    let response = Response::new(Body::from("Hello, Rust Web Server!"));
    Ok(response)
}

#[tokio::main]
async fn main() {
    let addr = ([127, 0, 0, 1], 8080).into();

    let make_service = make_service_fn(|_conn| {
        async { Ok::<_, hyper::Error>(service_fn(handle_request)) }
    });

    let server = Server::bind(&addr).serve(make_service);

    println!("Rust Web Server running on http://{}", addr);

    if let Err(e) = server.await {
        eprintln!("server error: {}", e);
    }
}

main()

Rust Web Server running on http://127.0.0.1:8080


#### Networked Chat Application

In [2]:
:dep serde = { version = "1.0.93", features = ["derive"] }

In [3]:
:dep serde_json = { version = "1.0.108" }

In [ ]:
use tokio::net::{TcpListener, TcpStream};
use tokio::io::{AsyncReadExt, AsyncWriteExt};
use tokio::sync::broadcast;
use serde::{Deserialize, Serialize};
use std::net::SocketAddr;

#[derive(Debug, Serialize, Deserialize, Clone)]
struct ChatMessage {
    username: String,
    content: String,
}

async fn handle_chat_client(
    stream: TcpStream,
    username: String,
    sender: broadcast::Sender<ChatMessage>,
) {
    println!("New client connected: {}", username);

    let mut receiver = sender.subscribe();

    let (mut reader, mut writer) = tokio::io::split(stream);

    tokio::spawn(async move {
        loop {
            let mut buffer = [0; 512];
            if let Ok(bytes_read) = reader.read(&mut buffer).await {
                if bytes_read == 0 {
                    break;
                }

                // Deserialize the received message
                if let Ok(message) = serde_json::from_slice::<ChatMessage>(&buffer[..bytes_read]) {
                    // Broadcast the message to all clients
                    sender.send(message.clone()).unwrap();
                }
            }
        }
    });

    while let Ok(message) = receiver.recv().await {
        // Exclude the current client from receiving its own messages
        if message.username != username {
            let response = serde_json::to_string(&message).unwrap();
            writer.write_all(response.as_bytes()).await.unwrap();
            println!("Sent message to {}: {}", username, message.content);
        }
    }

    println!("Client disconnected: {}", username);
}

#[tokio::main]
async fn main() {
    let (sender, _) = broadcast::channel::<ChatMessage>(100);

    let addr: SocketAddr = "127.0.0.1:8080".parse().unwrap();
    let listener = TcpListener::bind(&addr).await.unwrap();

    println!("Chat Server running on {}", addr);

    loop {
        let (stream, _) = listener.accept().await.unwrap();

        let mut buffer = [0; 512];
        let bytes_read = stream.peek(&mut buffer).await.unwrap();
        let username = String::from_utf8_lossy(&buffer[..bytes_read]).trim().to_owned();
        println!("Received connection request from: {}", username);

        tokio::spawn(handle_chat_client(stream, username.to_string(), sender.clone()));
    }
}

main()

Chat Server running on 127.0.0.1:8080
Received connection request from: {"username": "Mahmoud", "content": "Hello, server!"}
New client connected: {"username": "Mahmoud", "content": "Hello, server!"}
Sent message to {"username": "Mahmoud", "content": "Hello, server!"}: Hello, server!
Received connection request from: {"username": "Prime", "content": "Hello, server!"}
New client connected: {"username": "Prime", "content": "Hello, server!"}
Sent message to {"username": "Mahmoud", "content": "Hello, server!"}: Hello, server!
Sent message to {"username": "Prime", "content": "Hello, server!"}: Hello, server!


#### File Transfer Server

In [7]:
use std::net::{TcpListener, TcpStream};
use std::io::{Read, Write};
use std::fs::File;

fn handle_file_transfer_client(mut stream: TcpStream) {
    let mut filename_buffer = [0; 128];
    stream.read(&mut filename_buffer).unwrap();
    let filename = String::from_utf8_lossy(&filename_buffer)
        .split_terminator('\0')
        .next()
        .unwrap_or_default()
        .trim()
        .to_string();

    let mut file = File::create(&filename).unwrap();

    let mut buffer = [0; 1024];
    loop {
        let bytes_read = stream.read(&mut buffer).unwrap();
        if bytes_read == 0 {
            break;
        }
        file.write_all(&buffer[..bytes_read]).unwrap();
    }

    println!("File '{}' received successfully.", filename);
}

let listener = TcpListener::bind("127.0.0.1:8080").unwrap();
println!("File Transfer Server listening on 127.0.0.1:8080...");

for stream in listener.incoming() {
    match stream {
        Ok(stream) => {
            println!("Accepted a new connection for file transfer");
            std::thread::spawn(|| handle_file_transfer_client(stream));
        }
        Err(e) => {
            eprintln!("Error: {}", e);
        }
    }
}

File Transfer Server listening on 127.0.0.1:8080...
Accepted a new connection for file transfer
File 'file.txt' received successfully.
Accepted a new connection for file transfer


thread '<unnamed>' panicked at src/lib.rs:13:44:
called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }
stack backtrace:
   0: rust_begin_unwind
             at /rustc/cd674d61790607dfb6faa9d754bd3adfa13aea7c/library/std/src/panicking.rs:597:5
   1: core::panicking::panic_fmt
             at /rustc/cd674d61790607dfb6faa9d754bd3adfa13aea7c/library/core/src/panicking.rs:72:14
   2: core::result::unwrap_failed
             at /rustc/cd674d61790607dfb6faa9d754bd3adfa13aea7c/library/core/src/result.rs:1653:5
   3: ctx::handle_file_transfer_client
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Subprocess terminated with status: signal: 9 (SIGKILL)

#### Remote Command Execution Server

In [ ]:
use std::net::{TcpListener, TcpStream};
use std::io::{Read, Write};
use std::process::{Command, Stdio};

fn handle_remote_command_client(mut stream: TcpStream) {
    let mut command_buffer = [0; 512];
    stream.read(&mut command_buffer).unwrap();
    let command_str = String::from_utf8_lossy(&command_buffer)
        .split_terminator('\0')
        .next()
        .unwrap_or_default()
        .trim()
        .to_string();

    let output = Command::new("sh")
        .arg("-c")
        .arg(&command_str)
        .stdout(Stdio::piped())
        .output()
        .unwrap();

    stream.write_all(&output.stdout).unwrap();
}

let listener = TcpListener::bind("127.0.0.1:8080").unwrap();
println!("Remote Command Execution Server listening on 127.0.0.1:8080...");

for stream in listener.incoming() {
    match stream {
        Ok(stream) => {
            println!("Accepted a new connection for remote command execution");
            std::thread::spawn(|| handle_remote_command_client(stream));
        }
        Err(e) => {
            eprintln!("Error: {}", e);
        }
    }
}

Remote Command Execution Server listening on 127.0.0.1:8080...
Accepted a new connection for remote command execution
Accepted a new connection for remote command execution
Accepted a new connection for remote command execution


#### Peer-to-Peer File Sharing

In [ ]:
use std::net::{TcpListener, TcpStream};
use std::io::{Read, Write};
use std::fs::File;
use std::thread;
use std::env;

fn handle_p2p_client(mut stream: TcpStream) {
    let mut filename_buffer = [0; 128];
    stream.read(&mut filename_buffer).unwrap();

    let filename = String::from_utf8_lossy(&filename_buffer)
        .split_terminator('\0')
        .next()
        .unwrap_or_default()
        .trim()
        .to_string();

    println!("Server working directory: {:?}", env::current_dir());

    println!("Received request for file: {}", filename);

    let file = File::open(&filename);
    let response: Vec<u8> = match file {
        Ok(mut file) => {
            let mut buffer = Vec::new();
            file.read_to_end(&mut buffer).unwrap();
            buffer
        }
        Err(_) => {
            println!("File not found: {}", filename);
            Vec::new()
        }
    };

    stream.write_all(&response).unwrap();

    println!("Sent response to the client for file: {}", filename);
}

let listener = TcpListener::bind("127.0.0.1:8080").unwrap();
println!("P2P File Sharing Server listening on 127.0.0.1:8080...");

for stream in listener.incoming() {
    match stream {
        Ok(stream) => {
            println!("Accepted a new connection for P2P file sharing");
            thread::spawn(|| handle_p2p_client(stream));
        }
        Err(e) => {
            eprintln!("Error: {}", e);
        }
    }
}

P2P File Sharing Server listening on 127.0.0.1:8080...
Accepted a new connection for P2P file sharing
Server working directory: Ok("/home/mahmoud/Desktop/Rust Book/rust-lang-book/chapter_12")
Received request for file: file.txt
Sent response to the client for file: file.txt
Accepted a new connection for P2P file sharing
Server working directory: Ok("/home/mahmoud/Desktop/Rust Book/rust-lang-book/chapter_12")
Received request for file: file.txt
Sent response to the client for file: file.txt


#### Real-Time Collaborative Editing

In [6]:
:dep tokio-tungstenite = {version = "0.21.0"}

In [8]:
:dep futures-util = {version = "0.3.29"}

In [ ]:
use futures_util::StreamExt;
use futures_util::stream::TryStreamExt;
use tokio::net::TcpListener;
use tokio_tungstenite::accept_async;
use futures_util::SinkExt;

#[tokio::main]
async fn main() {
    let addr: std::net::SocketAddr = "127.0.0.1:8080".parse().expect("Invalid address");

    let listener = TcpListener::bind(&addr).await.expect("Failed to bind to address");
    println!("Real-Time Collaborative Editing Server listening on {}", addr);

    handle_editor_connection(listener).await;
}

async fn handle_editor_connection(stream: TcpListener) {
    while let Ok((socket, addr)) = stream.accept().await {
        println!("Accepted a new connection from: {}", addr);
        tokio::spawn(handle_editor_client(socket, addr));
    }
}

async fn handle_editor_client(socket: tokio::net::TcpStream, addr: std::net::SocketAddr) {
    let ws_stream = match accept_async(socket).await {
        Ok(ws_stream) => {
            println!("WebSocket connection established with: {}", addr);
            ws_stream
        }
        Err(e) => {
            eprintln!("Error during WebSocket handshake with {}: {}", addr, e);
            return;
        }
    };

    let (mut write, mut read) = ws_stream.split();

    tokio::spawn(async move {
        while let Ok(Some(message)) = read.try_next().await {
            println!("Received message from {}: {}", addr, message.to_text().unwrap_or_default());


            if let Err(e) = write.send(message).await {
                eprintln!("Error sending WebSocket message to {}: {}", addr, e);
                break;
            }
        }

        if let Err(e) = write.close().await {
            eprintln!("Error closing WebSocket connection with {}: {}", addr, e);
        }

        println!("WebSocket connection with {} closed", addr);
    });
}

main()

Real-Time Collaborative Editing Server listening on 127.0.0.1:8080
Accepted a new connection from: 127.0.0.1:41856
WebSocket connection established with: 127.0.0.1:41856
Received message from 127.0.0.1:41856: Hello, WebSocket server!

Accepted a new connection from: 127.0.0.1:42548
WebSocket connection established with: 127.0.0.1:42548
Received message from 127.0.0.1:42548: Hey all!



---
---